In [1]:
import pandas as pd
import numpy
from __future__ import division

In [2]:
df1_WS4T = pd.read_csv("/Users/asteriag/Desktop/DATASCIENCEPROJECT_WORKFLOW/TimeStamp_Engineering/engineering_OF_segment1_workflowS4_has_duration.csv",engine='python')

In [3]:
df1_WS4T.head(n=1)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,time_used_str,time_used_second
0,715808|405750|629365,Release Offer,Release Offer,8/12/2018 12:48:43,NaN,0.0


In [4]:
df1_WS4T['WRKFLW_TSK_MAP_GROUPED'].nunique()

68

In [5]:
df1_WS4T['WRKFLW_TSK_MAP_DETAILED'].nunique()


129

In [6]:
wd_count = df1_WS4T.groupby('WRKFLW_TSK_MAP_DETAILED').WRKFLW_TSK_MAP_DETAILED.count()
wd_dict = dict()
for i, v in wd_count.items():
    wd_dict[i] = v
sorted_wd_dict = sorted(wd_dict.items(), key = lambda x: x[1], reverse = True)
print(sorted_wd_dict)

[('Release Offer', 6293), ('Accept', 2354), ('Defer', 1200), ('OC - Satisfied - Evidence of course completion', 1154), ('AR - Satisfied - Overseas Student Health Cover details', 674), ('AR - Satisfied - Academic results for prior tertiary studies', 558), ('OC - Satisfied - English language requirements', 510), ('Manage Acceptance', 503), ('OC - Created - Evidence of course completion', 345), ('AR - Created - CONDITIONAL Acceptance agreement (International)', 293), ('AR - Satisfied - English language proficiency', 208), ('AR - Satisfied - Appointment of authorised representative', 128), ('Decline', 126), ('OC - Created - UMELBP (H3)', 123), ('OC - Satisfied - Overseas Student Health Cover details', 120), ('OC - Created - Overseas Student Health Cover details', 119), ('AR - Created - Overseas Student Health Cover details', 110), ('AR - Satisfied - Further information (1)', 109), ('AR - Satisfied - Current enrolment at the University of Melbourne', 105), ('AR - Created - Acceptance agreem

In [7]:
list = df1_WS4T.WRKFLW_TSK_MAP_DETAILED.unique()
avg_time_dict = dict()
for key in list :
    df_slides = df1_WS4T[df1_WS4T['WRKFLW_TSK_MAP_DETAILED'] == key]
    sum = df_slides['time_used_second'].sum()
    c = len(df_slides)
    avg = sum//c
    avg_time_dict[key] = avg

sorted_avg_time_dict = sorted(avg_time_dict.items(),key = lambda x: x[1], reverse = True)
print(sorted_avg_time_dict)

[('OC - Created - Offer condition (English language requirements)', 9006618.0), ('Send for Assessment', 8504475.0), ('AR - Satisfied - Syllabus (Melbourne School of Engineering)', 6010631.0), ('End My Assessment', 5915627.0), ('OC - Satisfied - Further information (1)', 5615932.0), ('AR - Reviewed - Further information (3)', 4730320.0), ('AR - Reviewed - Current enrolment at the University of Melbourne', 4575077.0), ('OC - Created - Evidence of course completion', 4427953.0), ('OC - Created - Further information (3)', 4293974.0), ('Release Offer', 3561079.0), ('AR - Reviewed - Official syllabus or subject descriptions (in English)', 3456300.0), ('Put Assessment On-Hold', 3427932.0), ('OC - Created - English language requirements', 3165166.0), ('OC - Reviewed - CDGDC transcript authentication for international students', 2813358.0), ('AR - Satisfied - Further information - Agent nomination form', 2701005.0), ('AR - Requested - Appointment of authorised representative', 2624146.0), ('AR 

In [8]:
numOfWorkflowProcesses = df1_WS4T.groupby('CASE_ID').CASE_ID.count()

count_dic=dict()
for i,v in numOfWorkflowProcesses.items():
    if count_dic.get(v) is None:
        count_dic[v]=[]
        count_dic[v].append(i)
    else:
        count_dic[v].append(i)
#count_dic
count_dic = dict(sorted(count_dic.items()))

In [9]:
len_list=[]
for key in count_dic.keys():
    len_list.append(len(count_dic[key]))

In [10]:
def strfdelta(tdelta, fmt):
    d=dict()
    d["days"],rem1 = divmod(tdelta,60*60*24)
    d["hours"], rem2 = divmod(rem1, 3600)
    d["minutes"], d["seconds"] = divmod(rem2, 60)
    return fmt.format(**d)

In [11]:
list_avgtime_per_key =[]
list_sumtime_row = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_person = 0
    for id in id_list:
        df = df1_WS4T[df1_WS4T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_person+=tsum
    list_avgtime_per_key.append(int((list_sumtime_person/len(count_dic[key]))))
    list_sumtime_row.append(list_sumtime_person)
list_avgtime_per_key_str = []
for i in list_avgtime_per_key:
    list_avgtime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [12]:
list_mediantime_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_per_person = []
    for id in id_list:
        df = df1_WS4T[df1_WS4T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_per_person.append(tsum)
    list_mediantime_per_key.append(numpy.median(list_sumtime_per_person))

list_mediantime_per_key_str = []
for i in list_mediantime_per_key:
    list_mediantime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [13]:
list_percent_reach_end = []
for key in count_dic.keys():
    id_list = count_dic[key]
    count_df_end_step = 0
    count_df_id = 0
    for id in id_list:
        df_id = df1_WS4T[df1_WS4T['CASE_ID']== id]
        df_end_step = df_id[df_id['WRKFLW_TSK_MAP_GROUPED'].isin(['Accept','Defer'])]
        count_df_id += df_id['CASE_ID'].nunique()
        count_df_end_step += df_end_step['CASE_ID'].nunique()
    list_percent_reach_end.append(str(round((count_df_end_step/count_df_id)*100,2))+ "%")

In [14]:
list_step_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = set()
    for id in id_list:
        df = df1_WS4T[df1_WS4T['CASE_ID']== id]
        for index, row in df.iterrows():
            list_step.add(row['WRKFLW_TSK_MAP_GROUPED'])
    list_step_per_key.append(list_step)

In [15]:
from collections import Counter
import collections, functools, operator
list_step_sum_time_per_key = []
list_step_count_per_key = []
list_step_ave_time_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = []
    step_time_list = []
    for id in id_list:
        df = df1_WS4T[df1_WS4T['CASE_ID']== id]
        step_time = dict(zip(df.WRKFLW_TSK_MAP_GROUPED, df.time_used_second))
        step_time_list.append(step_time)
        for index, row in df.iterrows():
            list_step.append(row['WRKFLW_TSK_MAP_GROUPED'])

    step_sum_time = dict(functools.reduce(operator.add,
         map(collections.Counter, step_time_list)))

    dict_list_step=dict(Counter(list_step))

    step_ave_time = {key: strfdelta(step_sum_time[key] // dict_list_step.get(key, 0),"{days} days {hours} hours {minutes} minutes {seconds} seconds")
                        for key in step_sum_time.keys()}

    sort_step_sum_time = sorted(step_sum_time.items(),key = lambda x: x[1], reverse = True)
    list_step_sum_time_per_key.append(sort_step_sum_time)

    sort_dict_list_step = sorted(dict_list_step.items(),key = lambda x: x[1], reverse = True)
    list_step_count_per_key.append(sort_dict_list_step[:6])

    sort_step_ave_time = sorted(step_ave_time.items(),key = lambda x: x[1], reverse = True)
    list_step_ave_time_per_key.append(sort_step_ave_time)

In [16]:
s1_t4=pd.DataFrame({'Number of Workflow processes in this Workflow Segment':count_dic.keys(),'Number of cases':len_list, 'average time for workflow segment':list_avgtime_per_key_str,'median time for workflow segment':list_mediantime_per_key_str,'percentage of cases reaching to end of workflow segment':list_percent_reach_end})
s1_t4['Num of Occurrences per step(descending)'] = list_step_count_per_key
s1_t4['Average time spends per step(descending)'] = list_step_ave_time_per_key
s1_t4['average time for workflow segment(seconds)'] = list_avgtime_per_key
s1_t4['sum time for workflow segment(seconds)'] = list_sumtime_row
s1_t4.head(n=10)


,Number of Workflow processes in this Workflow Segment,Number of cases,average time for workflow segment,median time for workflow segment,percentage of cases reaching to end of workflow segment,Num of Occurrences per step(descending),Average time spends per step(descending),average time for workflow segment(seconds),sum time for workflow segment(seconds)
0,1,1664,0 days 0 hours 0 minutes 0 seconds,0.0 days 0.0 hours 0.0 minutes 0.0 seconds,0.0%,"[(Release Offer, 1664)]",[],0,0.000000e+00
1,2,2114,63 days 16 hours 41 minutes 22 seconds,33.0 days 5.0 hours 25.0 minutes 43.0 seconds,73.08%,"[(Release Offer, 2117), (Defer, 815), (Accept,...","[(Release Offer, 63.0 days 14.0 hours 31.0 min...",5503282,1.163394e+10
2,3,1078,74 days 4 hours 42 minutes 6 seconds,47.0 days 1.0 hours 43.0 minutes 8.0 seconds,75.6%,"[(Release Offer, 1082), (Accept, 598), (OC - S...","[(Assessment Decision - No Decision, 9.0 days ...",6410526,6.910547e+09
3,4,590,83 days 2 hours 36 minutes 36 seconds,64.0 days 1.0 hours 38.0 minutes 13.0 seconds,80.51%,"[(Release Offer, 592), (Accept, 391), (OC - Sa...","[(End My Assessment, 92.0 days 15.0 hours 40.0...",7180596,4.236552e+09
4,5,379,104 days 13 hours 15 minutes 22 seconds,86.0 days 16.0 hours 18.0 minutes 6.0 seconds,87.86%,"[(Release Offer, 381), (OC - Satisfied - Other...","[(AR - Satisfied - ELR, 9.0 days 13.0 hours 34...",9033322,3.423629e+09
5,6,199,107 days 22 hours 50 minutes 0 seconds,89.0 days 17.0 hours 28.0 minutes 11.0 seconds,85.93%,"[(OC - Satisfied - Other, 222), (Release Offer...","[(AR - Created - Other, 9.0 days 22.0 hours 49...",9327000,1.856073e+09
6,7,118,123 days 11 hours 58 minutes 47 seconds,108.0 days 2.0 hours 28.0 minutes 16.0 seconds,88.14%,"[(OC - Satisfied - Other, 141), (Release Offer...","[(AR - Satisfied - financial (sponsorship, sch...",10670327,1.259099e+09
7,8,68,129 days 12 hours 56 minutes 12 seconds,124.0 days 14.0 hours 33.0 minutes 19.5 seconds,94.12%,"[(OC - Satisfied - Other, 92), (AR - Satisfied...","[(OC - Created - further information, 8.0 days...",11192172,7.610677e+08
8,9,36,121 days 9 hours 45 minutes 36 seconds,99.0 days 8.0 hours 50.0 minutes 33.0 seconds,88.89%,"[(OC - Satisfied - Other, 58), (Release Offer,...","[(AR - Created - further information, 8.0 days...",10489536,3.776233e+08
9,10,10,248 days 9 hours 23 minutes 17 seconds,264.0 days 6.0 hours 28.0 minutes 3.5 seconds,90.0%,"[(OC - Satisfied - Other, 19), (Release Offer,...","[(OC - Satisfied - Other, 7.0 days 2.0 hours 4...",21460997,2.146100e+08


In [17]:
s1_t4.to_csv('engineering_OF_segment1_workflowS4_table.csv', index=False)





